# Llama 3.2 3B fine tuning with large data set.

2025-03-10 17:46

Fixed Unsloth and Unsloth Zoo dependencies to specific commits because the head was soo unstable and caused failure to converge in the previous run (see 20250305-llama3b-large-dataset-failed).

This training run is ran for 6 days and the loss started to converge, did not change too much so I decided to stop the training. The epoch ran to about the Chinchilla suggested data size with regards to the total trainable parameters so this checks out. This indicates that a larger model is needed. Results are not great, a lot of repetition, not good perceptual loss. No consistent improvements in similarity and perceptual loss.

Lost connection to the notebook so to run the last 2 cells, I restarted the kernel and ran an extra step from the latest checkpoint (400). Running the 90B model next.

The data was limited to 50,000 tokens because that seems to be the 50th percentile of the large data set with regards to tokenized length and the 3B model can handle that with a single GPU. The limitation was mostly needed to trade data size for time, long sequences take a lot longer to train.

 

In [1]:
import unsloth
import os
import numpy as np
import pandas as pd

import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 50000

def load_model():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Llama-3.2-3B-unsloth-bnb-4bit",
        max_seq_length=max_seq_length,
        load_in_4bit=True,
        dtype=None,
    )
    
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        lora_alpha=16,
        lora_dropout=0,
        target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
        use_rslora=True,
        use_gradient_checkpointing="unsloth",
        random_state = 32,
        loftq_config = None,
    )
    return model, tokenizer

In [3]:
def create_trainer(model, tokenizer, training_data, max_steps):
    training_arguments = TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        auto_find_batch_size=True,
        gradient_accumulation_steps=16,
        num_train_epochs=40,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        # max_steps=max_steps,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=150,
        output_dir="output",
        seed=0,
        save_total_limit=3,
        
    )

    if max_steps is not None:
        training_arguments.max_steps = max_steps
    
    return SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=training_data,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=10,
        packing=True,
        args=training_arguments,
    )

In [4]:
import numpy as np
from utils.similarity import calculate_metrics
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torch

log_dir = 'output/runs'

def add_image_to_tensorboard(name, step, img_path):
    image = Image.open(img_path)
    image = image.convert('RGB')
    image_array = np.array(image)
    image_tensor = torch.from_numpy(image_array)
    image_tensor = image_tensor.permute(2, 0, 1)
    image_tensor = image_tensor.float() / 255.0
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_image(name, image_tensor, step)
    
def add_text_to_tensorboard(name, step, text):
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_text(name, text, step)

def postprocess_text(preds, labels):
    preds = [pred.strip().replace('<unk>', '') for pred in preds]
    labels = [[label.strip().replace('<unk>', '')] for label in labels]

    return preds, labels

def compute_metrics(decoded_predictions, decoded_labels, steps):
    similarity_scores = []
    perceptual_losses = []
    index = 1
    
    for prediction, label in zip(decoded_predictions, decoded_labels):
        prediction = prediction.replace(tokenizer.eos_token, '')
        
        add_text_to_tensorboard(f'valid_{index}_label_text', steps, label)
        add_text_to_tensorboard(f'valid_{index}_prediction_text', steps, prediction)
        
        metrics = calculate_metrics(prediction, label)
        
        if metrics is not None:
            similarity_scores.append(metrics['similarity'])
            perceptual_losses.append(metrics['perceptual_loss'])
            
            add_image_to_tensorboard(f'valid_{index}_expectation', steps, metrics['expected_screenshot_path'])
            add_image_to_tensorboard(f'valid_{index}_prediction', steps, metrics['predicted_screenshot_path'])
        
        index += 1

    results = {
        "similarity": float(np.mean(similarity_scores)),
        "perceptual_loss": float(np.mean(perceptual_losses)),
    }
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_scalar('similarity', results['similarity'], steps)
    writer.add_scalar('perceptual_loss', results['perceptual_loss'], steps)
    
    print("Similarity:", results['similarity'])
    print("Perceptual loss:", results['perceptual_loss'])

    return results

def test_prediction(model, data, steps):
    answers = []
    labels = []
    print("Generating predictions...")
    for row in data:
        inputs = tokenizer(
        [
            data_prompt.format(
                #instructions
                row['svg'],
                #answer
                "",
            )
        ], return_tensors = "pt").to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens = max_seq_length, use_cache = True)
        answer = tokenizer.batch_decode(outputs)
        answers.append(answer[0].split("### Response:")[-1])
        labels.append(row['html'])

    print("Computing metrics...")
    return compute_metrics(answers, labels, steps)

In [5]:
!rm -rf output

In [ ]:
!apt install zip -y
!rm -rf data-rb-large
!mkdir -p data-rb-large
!wget "https://www.dropbox.com/scl/fi/868hi141d4hrzwntxxih3/data-rb-large-with-tokenizer.zip?rlkey=md8iq2htminafzqcol3tzfbva&dl=1" -O model.zip
!unzip model.zip -d data-rb-large

!rm -rf data-rb-validate
!mkdir -p data-rb-validate
!wget "https://www.dropbox.com/scl/fi/5szml8y5l248mcabj9rqg/verify-dataset.zip?rlkey=se33rwtxgngn0ts1i0pc8f6wk&st=1d68x9zt&dl=1" -O validate.zip
!unzip validate.zip -d data-rb-validate

In [ ]:
from datasets import load_from_disk, DatasetDict
dataset = load_from_disk('data-rb-large')
visual_validation_dataset = load_from_disk('data-rb-validate')

train_test_split = dataset.train_test_split(test_size=100/len(dataset)) 
train_dataset = train_test_split["train"]
temp_dataset = train_test_split["test"]

dataset = DatasetDict({
    "train": train_test_split["train"],
    "valid": visual_validation_dataset,
    "test": train_test_split["test"],
})

In [ ]:
dataset

In [5]:
model, tokenizer = load_model()

data_prompt = """Your job is to take an SVG file of a web design and convert it into a pixel-perfect HTML and CSS markup and stylesheet.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs       = examples["svg"]
    outputs      = examples["html"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }



==((====))==  Unsloth 2025.3.3: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    NVIDIA H100 NVL. Num GPUs = 1. Max memory: 93.111 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
training_data = dataset.map(formatting_prompt, batched=True)

In [ ]:
training_data

In [ ]:
def get_token_lengths(examples):
    inputs = tokenizer(
        examples['text'],
        truncation=False,  # Don't truncate yet
        padding=False,     # Don't pad yet
        return_length=True,
    )

    return inputs

tokenized_data = training_data.map(get_token_lengths, batched=True)

def filter_function(example):
    return example['length'] <= max_seq_length

filtered_data = tokenized_data.filter(filter_function)

print(filtered_data)

In [ ]:
filtered_data = filtered_data.remove_columns(["input_ids", "attention_mask", "length"])
filtered_data.save_to_disk('data-rb-large-filtered-' + str(max_seq_length))

In [6]:
from datasets import load_from_disk

filtered_data = load_from_disk('data-rb-large-filtered-' + str(max_seq_length))

filtered_data

Loading dataset from disk:   0%|          | 0/190 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 293102
    })
    valid: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 4
    })
    test: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 65
    })
})

In [7]:
import torch
from tqdm import tqdm

# resume = False
resume = True

# for steps in tqdm(range(0, 1500, 50)):
for steps in tqdm(range(400, 450, 50)):
    print(f"Steps: {steps}")

    if steps > 0:
        trainer = create_trainer(model, tokenizer, filtered_data['train'], steps)
        if resume:
            trainer.train(resume_from_checkpoint=True)
        else:
            trainer.train()
            resume = True
        
    model = FastLanguageModel.for_inference(model)

    results = test_prediction(model, filtered_data['valid'], steps)

    if results is not None and results['perceptual_loss'] == 0.0:
        break

    model = FastLanguageModel.for_training(model)

    

  0%|          | 0/1 [00:00<?, ?it/s]

Steps: 400


Loading dataset shards:   0%|          | 0/110 [00:00<?, ?it/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 109,974 | Num Epochs = 1 | Total steps = 400
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 16 x 1) = 64
 "-____-"     Trainable parameters = 24,313,856/1,865,526,272 (1.30% trained)
	per_device_train_batch_size: 8 (from args) != 4 (from trainer_state.json)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
401,0.433000


Generating predictions...
Computing metrics...


100%|██████████| 1/1 [1:05:10<00:00, 3910.77s/it]

Similarity: 0.3831438615918159
Perceptual loss: 0.6980251669883728



----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 33920)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/utils/similarity.py", line 110, in __init__
    super().__init__(*args, directory=VALIDATION_DATA_DIR, **kwargs)
  File "/opt/conda/lib/python3.11/http/server.py", line 671, in __init__
    super().__init__(*args, **kwargs)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/opt/conda/lib/python3.11/http/server.py", line 436, in handle
    self.handle_one_request()
  File "/opt/c

In [8]:
test_index = 0
text = filtered_data['test'][test_index]['svg']
model = FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = max_seq_length, use_cache = True)
answer=tokenizer.batch_decode(outputs)
answer = answer[0].split("### Response:")[-1]

print(filtered_data['test'][test_index]['svg'])
print(filtered_data['test'][test_index]['html'])
print("Answer of the question is:", answer)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="884" viewBox="0 0 393 884"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="--color-scheme-wireframe1" class="--color-scheme-wireframe --template-the-o-g-profile identity-body tw-bg-template-background" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="turbo-cable-stream-source1 div1 profile_hover_card main-content style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="393" height="916" x="0" y="0" fill="rgb(255, 255, 255)"/></g><g data-stacking-layer="childStackingContextsWithPositiveStackLevels"><g data-tag="div" id="profile_hover_card" class="popover-profile-card-container" data-z-index="1070" data-stacking-context="true"/></g><g data-tag="turbo-cable-stream-source" id

In [9]:
test_prediction(model, filtered_data['test'], steps+1)

Generating predictions...
Computing metrics...
Script failed with error: Traceback (most recent call last):
  File "/utils/similarity.py", line 188, in <module>
    take_screenshot(args.predicted_url, args.predicted_screenshot_path)
  File "/utils/similarity.py", line 97, in take_screenshot
    page.goto(url)
  File "/opt/conda/lib/python3.11/site-packages/playwright/sync_api/_generated.py", line 9018, in goto
    self._sync(
  File "/opt/conda/lib/python3.11/site-packages/playwright/_impl/_sync_base.py", line 115, in _sync
    return task.result()
           ^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/playwright/_impl/_page.py", line 551, in goto
    return await self._main_frame.goto(**locals_to_params(locals()))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/playwright/_impl/_frame.py", line 145, in goto
    await self._channel.send("goto", locals_to_params(locals()))
  File "/opt/conda/lib/pytho

{'similarity': 0.9132825120032163, 'perceptual_loss': 0.3930639995596721}